In [1]:
# Import Library
import pandas as pd
import re


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait

from selenium.common.exceptions import TimeoutException



In [4]:
# Scrape QBs
df=pd.DataFrame(columns=['Unnamed: 0_level_0_Player', 'Unnamed: 1_level_0_Game',
       'Unnamed: 2_level_0_Pts*', 'Passing_Att', 'Passing_Cmp', 'Passing_Yds',
       'Passing_TD', 'Passing_Int', 'Passing_2Pt', 'Rushing_Att',
       'Rushing_Yds', 'Rushing_TD', 'Rushing_2Pt', 'Receiving_Rec',
       'Receiving_Yds', 'Receiving_TD', 'Receiving_2Pt', 'Fumbles_FL',
       'Fumbles_TD'])
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

for j in range(1, 2):

    season = 2022
    week = j

    # Scrape web for stats
    url = f"https://www.footballdb.com/fantasy-football/index.html?pos=QB&yr={season}&wk={week}&key=48ca46aa7d721af4d58dccc0c249a1c4"

    PATH = "/Applications/chromedriver"
    driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
    wait = WebDriverWait(driver, 20)
    driver.get(url)

    wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='leftcol']/div[3]/table")))
    driver.execute_script("window.stop();")

    result = pd.read_html(driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table").get_attribute("outerHTML"))
    regex_result = driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table/tbody").get_attribute("outerHTML")

    # find player's team based on which team is bolded
    regex_teams = re.findall("<b>(.*?)</b>", regex_result)

    # flatten multiindex
    result = result[0]
    result.columns = result.columns.get_level_values(0) + '_' +  result.columns.get_level_values(1)

    # Set values for new columns
    result['team'] = regex_teams
    result['season'] = season
    result['week'] = week

    # concatenate to master df
    df = pd.concat([df, result], axis=0, ignore_index=True)

df1 = df

,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_Game,Unnamed: 2_level_0_Pts*,Passing_Att,Passing_Cmp,Passing_Yds,Passing_TD,Passing_Int,Passing_2Pt,Rushing_Att,...,Rushing_2Pt,Receiving_Rec,Receiving_Yds,Receiving_TD,Receiving_2Pt,Fumbles_FL,Fumbles_TD,team,season,week
0,Patrick MahomesP. Mahomes,KC@ARI,44.0,39,30,360,5,0,0,3,...,0,0,0,0,0,0,0,KC,2022.0,1.0
1,Josh AllenJ. Allen,BUF@LA,36.0,31,26,297,3,2,0,10,...,0,0,0,0,0,0,0,BUF,2022.0,1.0
2,Carson WentzC. Wentz,JAX@WAS,35.0,41,27,313,4,2,1,6,...,0,0,0,0,0,0,0,WAS,2022.0,1.0
3,Justin HerbertJ. Herbert,LV@LAC,29.0,34,26,279,3,0,0,4,...,0,0,0,0,0,0,0,LAC,2022.0,1.0
4,Lamar JacksonL. Jackson,BAL@NYJ,25.0,30,17,213,3,1,0,6,...,0,0,0,0,0,0,0,BAL,2022.0,1.0
5,Jalen HurtsJ. Hurts,PHI@DET,24.0,32,18,243,0,0,0,17,...,0,0,0,0,0,0,0,PHI,2022.0,1.0
6,Jameis WinstonJ. Winston,NO@ATL,24.0,34,23,269,2,0,1,2,...,0,0,0,0,0,0,0,NO,2022.0,1.0
7,Kirk CousinsK. Cousins,GB@MIN,23.0,32,23,277,2,0,0,0,...,0,0,0,0,0,0,0,MIN,2022.0,1.0
8,Kyler MurrayK. Murray,KC@ARI,23.0,34,22,193,2,0,1,5,...,0,0,0,0,0,0,0,ARI,2022.0,1.0
9,Joe BurrowJ. Burrow,PIT@CIN,22.0,53,33,338,2,4,1,6,...,0,0,0,0,0,1,0,CIN,2022.0,1.0


In [17]:
# Clean QBs
df1 = df1.rename(columns={'Unnamed: 0_level_0_Player':'player', 'Unnamed: 1_level_0_Game':'opponent',
       'Passing_Att':'passA', 'Passing_Cmp':"passC", 'Passing_Yds':"passY",
       'Passing_TD':'passT', 'Passing_Int':'passI', 'Passing_2Pt':'pass2', 'Rushing_Att':'rushA',
       'Rushing_Yds':'rushY', 'Rushing_TD':'rushT', 'Rushing_2Pt':'rush2', 'Receiving_Rec':'recC',
       'Receiving_Yds':'recY', 'Receiving_TD':'recT', 'Receiving_2Pt':'rec2', 'Fumbles_FL':'fum', 'Fumbles_TD':'fumT'
       })
df1 = df1[['player', 'season', 'week', 'team', 'opponent', 'passA', 'passC', 'passY', 'passT', 'passI', 'pass2',
       'rushA', 'rushY', 'rushT', 'rush2', 'recC', 'recY', 'recT', 'rec2',
       'fum']]
df1[['tm1', 'tm2']] = df1['opponent'].str.split('@', expand=True)

def teamPicker(row):
    team = row['team']
    if row['tm1']==team:
        opp = row['tm2']
    else:
        opp = row['tm1']
    return opp
df1['opponent'] = df1.apply(teamPicker, axis=1)

df1 = df1.drop(columns=['tm1', 'tm2'])

df1[['player', 'todrop']] = df1['player'].str.split('.\xa0', expand=True)
df1 = df1.drop(columns=['todrop'])

def dropLast(name):
    name = name[:-1]
    return name
df1['player'] = df1['player'].apply(dropLast)

df1['pos'] = 'QB'

df1['posRank'] = 'NA'
for i in range(len(df1)):
    team = df1.iloc[i]['team']
    season = df1.iloc[i]['season']
    week = df1.iloc[i]['week']
    player = df1.iloc[i]['player']
    slicedf = df1.loc[(df1['team']==team) & (df1['season']==season) & (df1['week']==week)]
    slicedf = slicedf.sort_values(by=['passA'], ascending=False, ignore_index=True)
    posRank = slicedf.index[slicedf['player'] == player].to_list()[0]
    posRank += 1
    posRank = str(posRank)
    df1.at[i, 'posRank'] = posRank
df1['posRank'] = df1['pos'] + df1['posRank']
df1

,player,season,week,team,opponent,passA,passC,passY,passT,passI,...,rushY,rushT,rush2,recC,recY,recT,rec2,fum,pos,posRank
0,Patrick Mahomes,2022.0,1.0,KC,ARI,39,30,360,5,0,...,5,0,0,0,0,0,0,0,QB,QB1
1,Josh Allen,2022.0,1.0,BUF,LA,31,26,297,3,2,...,56,1,0,0,0,0,0,0,QB,QB1
2,Carson Wentz,2022.0,1.0,WAS,JAX,41,27,313,4,2,...,12,0,0,0,0,0,0,0,QB,QB1
3,Justin Herbert,2022.0,1.0,LAC,LV,34,26,279,3,0,...,1,0,0,0,0,0,0,0,QB,QB1
4,Lamar Jackson,2022.0,1.0,BAL,NYJ,30,17,213,3,1,...,17,0,0,0,0,0,0,0,QB,QB1
5,Jalen Hurts,2022.0,1.0,PHI,DET,32,18,243,0,0,...,90,1,0,0,0,0,0,0,QB,QB1
6,Jameis Winston,2022.0,1.0,NO,ATL,34,23,269,2,0,...,9,0,0,0,0,0,0,0,QB,QB1
7,Kirk Cousins,2022.0,1.0,MIN,GB,32,23,277,2,0,...,0,0,0,0,0,0,0,0,QB,QB1
8,Kyler Murray,2022.0,1.0,ARI,KC,34,22,193,2,0,...,29,0,0,0,0,0,0,0,QB,QB1
9,Joe Burrow,2022.0,1.0,CIN,PIT,53,33,338,2,4,...,47,0,0,0,0,0,0,1,QB,QB1


In [6]:
# Scrape RBs
df=pd.DataFrame(columns=['Unnamed: 0_level_0_Player', 'Unnamed: 1_level_0_Game',
       'Unnamed: 2_level_0_Pts*', 'Passing_Att', 'Passing_Cmp', 'Passing_Yds',
       'Passing_TD', 'Passing_Int', 'Passing_2Pt', 'Rushing_Att',
       'Rushing_Yds', 'Rushing_TD', 'Rushing_2Pt', 'Receiving_Rec',
       'Receiving_Yds', 'Receiving_TD', 'Receiving_2Pt', 'Fumbles_FL',
       'Fumbles_TD'])
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

for j in range(1, 2):

    season = 2022
    week = j
    
    # Scrape web for stats
    url = f"https://www.footballdb.com/fantasy-football/index.html?pos=RB&yr={season}&wk={week}&key=48ca46aa7d721af4d58dccc0c249a1c4"

    PATH = "/Applications/chromedriver"
    driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
    wait = WebDriverWait(driver, 20)
    driver.get(url)

    wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='leftcol']/div[3]/table")))
    driver.execute_script("window.stop();")

    result = pd.read_html(driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table").get_attribute("outerHTML"))
    regex_result = driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table/tbody").get_attribute("outerHTML")

    # find player's team based on which team is bolded
    regex_teams = re.findall("<b>(.*?)</b>", regex_result)

    # flatten multiindex
    result = result[0]
    result.columns = result.columns.get_level_values(0) + '_' +  result.columns.get_level_values(1)

    # Set values for new columns
    result['team'] = regex_teams
    result['season'] = season
    result['week'] = week

    # concatenate to master df
    df = pd.concat([df, result], axis=0, ignore_index=True)

df2 = df

,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_Game,Unnamed: 2_level_0_Pts*,Passing_Att,Passing_Cmp,Passing_Yds,Passing_TD,Passing_Int,Passing_2Pt,Rushing_Att,...,Rushing_2Pt,Receiving_Rec,Receiving_Yds,Receiving_TD,Receiving_2Pt,Fumbles_FL,Fumbles_TD,team,season,week
0,Saquon BarkleyS. Barkley,NYG@TEN,27.0,0,0,0,0,0,0,18,...,0,6,30,0,1,0,0,NYG,2022.0,1.0
1,D'Andre SwiftD. Swift,PHI@DET,23.0,0,0,0,0,0,0,15,...,0,3,31,0,0,0,0,DET,2022.0,1.0
2,Jonathan TaylorJ. Taylor,IND@HOU,23.0,0,0,0,0,0,0,31,...,0,4,14,0,0,0,0,IND,2022.0,1.0
3,Clyde Edwards-HelaireC. Edwards-Helaire,KC@ARI,19.0,0,0,0,0,0,0,7,...,0,3,32,2,0,0,0,KC,2022.0,1.0
4,Cordarrelle PattersonC. Patterson,NO@ATL,19.0,0,0,0,0,0,0,22,...,0,3,16,0,0,0,0,ATL,2022.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Patrick RicardP. Ricard,BAL@NYJ,0.0,0,0,0,0,0,0,0,...,0,1,7,0,0,0,0,BAL,2022.0,1.0
76,Jaylen WarrenJ. Warren,PIT@CIN,0.0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,PIT,2022.0,1.0
77,Avery WilliamsA. Williams,NO@ATL,0.0,0,0,0,0,0,0,2,...,0,1,8,0,0,0,0,ATL,2022.0,1.0
78,Damien WilliamsD. Williams,NO@ATL,0.0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,ATL,2022.0,1.0


In [18]:
df2 = df2.rename(columns={'Unnamed: 0_level_0_Player':'player', 'Unnamed: 1_level_0_Game':'opponent',
       'Passing_Att':'passA', 'Passing_Cmp':"passC", 'Passing_Yds':"passY",
       'Passing_TD':'passT', 'Passing_Int':'passI', 'Passing_2Pt':'pass2', 'Rushing_Att':'rushA',
       'Rushing_Yds':'rushY', 'Rushing_TD':'rushT', 'Rushing_2Pt':'rush2', 'Receiving_Rec':'recC',
       'Receiving_Yds':'recY', 'Receiving_TD':'recT', 'Receiving_2Pt':'rec2', 'Fumbles_FL':'fum', 'Fumbles_TD':'fumT'
       })
df2 = df2[['player', 'season', 'week', 'team', 'opponent', 'passA', 'passC', 'passY', 'passT', 'passI', 'pass2',
       'rushA', 'rushY', 'rushT', 'rush2', 'recC', 'recY', 'recT', 'rec2',
       'fum']]
df2[['tm1', 'tm2']] = df2['opponent'].str.split('@', expand=True)

def teamPicker(row):
    team = row['team']
    if row['tm1']==team:
        opp = row['tm2']
    else:
        opp = row['tm1']
    return opp
df2['opponent'] = df2.apply(teamPicker, axis=1)

df2 = df2.drop(columns=['tm1', 'tm2'])

df2[['player', 'todrop']] = df2['player'].str.split('.\xa0', expand=True)
df2 = df2.drop(columns=['todrop'])

def dropLast(name):
    name = name[:-1]
    return name
df2['player'] = df2['player'].apply(dropLast)

df2['pos'] = 'RB'

df2['posRank'] = 'NA'
for i in range(len(df2)):
    team = df2.iloc[i]['team']
    season = df2.iloc[i]['season']
    week = df2.iloc[i]['week']
    player = df2.iloc[i]['player']
    slicedf = df2.loc[(df2['team']==team) & (df2['season']==season) & (df2['week']==week)]
    slicedf = slicedf.sort_values(by=['rushA'], ascending=False, ignore_index=True)
    posRank = slicedf.index[slicedf['player'] == player].to_list()[0]
    posRank += 1
    posRank = str(posRank)
    df2.at[i, 'posRank'] = posRank
df2['posRank'] = df2['pos'] + df2['posRank']
df2

,player,season,week,team,opponent,passA,passC,passY,passT,passI,...,rushY,rushT,rush2,recC,recY,recT,rec2,fum,pos,posRank
0,Saquon Barkley,2022.0,1.0,NYG,TEN,0,0,0,0,0,...,164,1,0,6,30,0,1,0,RB,RB1
1,D'Andre Swift,2022.0,1.0,DET,PHI,0,0,0,0,0,...,144,1,0,3,31,0,0,0,RB,RB1
2,Jonathan Taylor,2022.0,1.0,IND,HOU,0,0,0,0,0,...,161,1,0,4,14,0,0,0,RB,RB1
3,Clyde Edwards-Helaire,2022.0,1.0,KC,ARI,0,0,0,0,0,...,42,0,0,3,32,2,0,0,RB,RB2
4,Cordarrelle Patterson,2022.0,1.0,ATL,NO,0,0,0,0,0,...,120,1,0,3,16,0,0,0,RB,RB1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Patrick Ricard,2022.0,1.0,BAL,NYJ,0,0,0,0,0,...,0,0,0,1,7,0,0,0,RB,RB4
76,Jaylen Warren,2022.0,1.0,PIT,CIN,0,0,0,0,0,...,7,0,0,0,0,0,0,0,RB,RB2
77,Avery Williams,2022.0,1.0,ATL,NO,0,0,0,0,0,...,7,0,0,1,8,0,0,0,RB,RB2
78,Damien Williams,2022.0,1.0,ATL,NO,0,0,0,0,0,...,2,0,0,0,0,0,0,0,RB,RB3


In [8]:
# Scrape WRs
df=pd.DataFrame(columns=['Unnamed: 0_level_0_Player', 'Unnamed: 1_level_0_Game',
       'Unnamed: 2_level_0_Pts*', 'Passing_Att', 'Passing_Cmp', 'Passing_Yds',
       'Passing_TD', 'Passing_Int', 'Passing_2Pt', 'Rushing_Att',
       'Rushing_Yds', 'Rushing_TD', 'Rushing_2Pt', 'Receiving_Rec',
       'Receiving_Yds', 'Receiving_TD', 'Receiving_2Pt', 'Fumbles_FL',
       'Fumbles_TD'])
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

for j in range(1, 2):

    season = 2022
    week = j
    
    # Scrape web for stats
    url = f"https://www.footballdb.com/fantasy-football/index.html?pos=WR&yr={season}&wk={week}&key=48ca46aa7d721af4d58dccc0c249a1c4"

    PATH = "/Applications/chromedriver"
    driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
    wait = WebDriverWait(driver, 20)
    driver.get(url)

    wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='leftcol']/div[3]/table")))
    driver.execute_script("window.stop();")

    result = pd.read_html(driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table").get_attribute("outerHTML"))
    regex_result = driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table/tbody").get_attribute("outerHTML")

    # find player's team based on which team is bolded
    regex_teams = re.findall("<b>(.*?)</b>", regex_result)

    # flatten multiindex
    result = result[0]
    result.columns = result.columns.get_level_values(0) + '_' +  result.columns.get_level_values(1)

    # Set values for new columns
    result['team'] = regex_teams
    result['season'] = season
    result['week'] = week

    # concatenate to master df
    df = pd.concat([df, result], axis=0, ignore_index=True)

df3 = df

,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_Game,Unnamed: 2_level_0_Pts*,Passing_Att,Passing_Cmp,Passing_Yds,Passing_TD,Passing_Int,Passing_2Pt,Rushing_Att,...,Rushing_2Pt,Receiving_Rec,Receiving_Yds,Receiving_TD,Receiving_2Pt,Fumbles_FL,Fumbles_TD,team,season,week
0,Justin JeffersonJ. Jefferson,GB@MIN,30.0,0,0,0,0,0,0,0,...,0,9,184,2,0,0,0,MIN,2022.0,1.0
1,Davante AdamsD. Adams,LV@LAC,20.0,0,0,0,0,0,0,0,...,0,10,141,1,0,0,0,LV,2022.0,1.0
2,Ja'Marr ChaseJ. Chase,PIT@CIN,18.0,0,0,0,0,0,0,0,...,0,10,129,1,0,0,0,CIN,2022.0,1.0
3,Stefon DiggsS. Diggs,BUF@LA,18.0,0,0,0,0,0,0,0,...,0,8,122,1,0,0,0,BUF,2022.0,1.0
4,Cooper KuppC. Kupp,BUF@LA,18.0,0,0,0,0,0,0,0,...,0,13,128,1,0,0,0,LA,2022.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Zach PascalZ. Pascal,PHI@DET,0.0,0,0,0,0,0,0,0,...,0,1,7,0,0,0,0,PHI,2022.0,1.0
116,Breshad PerrimanB. Perriman,TB@DAL,0.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,TB,2022.0,1.0
117,George PickensG. Pickens,PIT@CIN,0.0,0,0,0,0,0,0,0,...,0,1,3,0,0,0,0,PIT,2022.0,1.0
118,Wan'Dale RobinsonW. Robinson,NYG@TEN,0.0,0,0,0,0,0,0,0,...,0,1,5,0,0,0,0,NYG,2022.0,1.0


In [19]:
# Clean WRs
df3 = df3.rename(columns={'Unnamed: 0_level_0_Player':'player', 'Unnamed: 1_level_0_Game':'opponent',
       'Passing_Att':'passA', 'Passing_Cmp':"passC", 'Passing_Yds':"passY",
       'Passing_TD':'passT', 'Passing_Int':'passI', 'Passing_2Pt':'pass2', 'Rushing_Att':'rushA',
       'Rushing_Yds':'rushY', 'Rushing_TD':'rushT', 'Rushing_2Pt':'rush2', 'Receiving_Rec':'recC',
       'Receiving_Yds':'recY', 'Receiving_TD':'recT', 'Receiving_2Pt':'rec2', 'Fumbles_FL':'fum', 'Fumbles_TD':'fumT'
       })
df3 = df3[['player', 'season', 'week', 'team', 'opponent', 'passA', 'passC', 'passY', 'passT', 'passI', 'pass2',
       'rushA', 'rushY', 'rushT', 'rush2', 'recC', 'recY', 'recT', 'rec2',
       'fum']]
df3[['tm1', 'tm2']] = df3['opponent'].str.split('@', expand=True)

def teamPicker(row):
    team = row['team']
    if row['tm1']==team:
        opp = row['tm2']
    else:
        opp = row['tm1']
    return opp
df3['opponent'] = df3.apply(teamPicker, axis=1)

df3 = df3.drop(columns=['tm1', 'tm2'])

df3[['player', 'todrop']] = df3['player'].str.split('.\xa0', expand=True)
df3 = df3.drop(columns=['todrop'])

def dropLast(name):
    name = name[:-1]
    return name
df3['player'] = df3['player'].apply(dropLast)

df3['pos'] = 'WR'

df3['posRank'] = 'NA'
for i in range(len(df3)):
    team = df3.iloc[i]['team']
    season = df3.iloc[i]['season']
    week = df3.iloc[i]['week']
    player = df3.iloc[i]['player']
    slicedf = df3.loc[(df3['team']==team) & (df3['season']==season) & (df3['week']==week)]
    slicedf = slicedf.sort_values(by=['recC'], ascending=False, ignore_index=True)
    posRank = slicedf.index[slicedf['player'] == player].to_list()[0]
    posRank += 1
    posRank = str(posRank)
    df3.at[i, 'posRank'] = posRank
df3['posRank'] = df3['pos'] + df3['posRank']
df3

,player,season,week,team,opponent,passA,passC,passY,passT,passI,...,rushY,rushT,rush2,recC,recY,recT,rec2,fum,pos,posRank
0,Justin Jefferson,2022.0,1.0,MIN,GB,0,0,0,0,0,...,0,0,0,9,184,2,0,0,WR,WR1
1,Davante Adams,2022.0,1.0,LV,LAC,0,0,0,0,0,...,0,0,0,10,141,1,0,0,WR,WR1
2,Ja'Marr Chase,2022.0,1.0,CIN,PIT,0,0,0,0,0,...,0,0,0,10,129,1,0,0,WR,WR1
3,Stefon Diggs,2022.0,1.0,BUF,LA,0,0,0,0,0,...,0,0,0,8,122,1,0,0,WR,WR1
4,Cooper Kupp,2022.0,1.0,LA,BUF,0,0,0,0,0,...,0,0,0,13,128,1,0,0,WR,WR1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Zach Pascal,2022.0,1.0,PHI,DET,0,0,0,0,0,...,0,0,0,1,7,0,0,0,WR,WR2
116,Breshad Perriman,2022.0,1.0,TB,DAL,0,0,0,0,0,...,-7,0,0,0,0,0,0,0,WR,WR6
117,George Pickens,2022.0,1.0,PIT,CIN,0,0,0,0,0,...,0,0,0,1,3,0,0,0,WR,WR3
118,Wan'Dale Robinson,2022.0,1.0,NYG,TEN,0,0,0,0,0,...,0,0,0,1,5,0,0,0,WR,WR5


In [10]:
# Scrape TEs
df=pd.DataFrame(columns=['Unnamed: 0_level_0_Player', 'Unnamed: 1_level_0_Game',
       'Unnamed: 2_level_0_Pts*', 'Passing_Att', 'Passing_Cmp', 'Passing_Yds',
       'Passing_TD', 'Passing_Int', 'Passing_2Pt', 'Rushing_Att',
       'Rushing_Yds', 'Rushing_TD', 'Rushing_2Pt', 'Receiving_Rec',
       'Receiving_Yds', 'Receiving_TD', 'Receiving_2Pt', 'Fumbles_FL',
       'Fumbles_TD'])
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

for j in range(1, 2):

    season = 2022
    week = j
    
    # Scrape web for stats
    url = f"https://www.footballdb.com/fantasy-football/index.html?pos=TE&yr={season}&wk={week}&key=48ca46aa7d721af4d58dccc0c249a1c4"

    PATH = "/Applications/chromedriver"
    driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
    wait = WebDriverWait(driver, 20)
    driver.get(url)

    wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='leftcol']/div[3]/table")))
    driver.execute_script("window.stop();")

    result = pd.read_html(driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table").get_attribute("outerHTML"))
    regex_result = driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table/tbody").get_attribute("outerHTML")

    # find player's team based on which team is bolded
    regex_teams = re.findall("<b>(.*?)</b>", regex_result)

    # flatten multiindex
    result = result[0]
    result.columns = result.columns.get_level_values(0) + '_' +  result.columns.get_level_values(1)

    # Set values for new columns
    result['team'] = regex_teams
    result['season'] = season
    result['week'] = week

    # concatenate to master df
    df = pd.concat([df, result], axis=0, ignore_index=True)

df4 = df

,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_Game,Unnamed: 2_level_0_Pts*,Passing_Att,Passing_Cmp,Passing_Yds,Passing_TD,Passing_Int,Passing_2Pt,Rushing_Att,...,Rushing_2Pt,Receiving_Rec,Receiving_Yds,Receiving_TD,Receiving_2Pt,Fumbles_FL,Fumbles_TD,team,season,week
0,Travis KelceT. Kelce,KC@ARI,18.0,0,0,0,0,0,0,0,...,0,8,121,1,0,0,0,KC,2022.0,1.0
1,O.J. HowardO. Howard,IND@HOU,15.0,0,0,0,0,0,0,0,...,0,2,38,2,0,0,0,HOU,2022.0,1.0
2,Taysom HillT. Hill,NO@ATL,14.0,0,0,0,0,0,0,4,...,0,1,2,0,0,0,0,NO,2022.0,1.0
3,Gerald EverettG. Everett,LV@LAC,11.0,0,0,0,0,0,0,0,...,0,3,54,1,0,0,0,LAC,2022.0,1.0
4,Will DisslyW. Dissly,DEN@SEA,10.0,0,0,0,0,0,0,0,...,0,3,43,1,0,0,0,SEA,2022.0,1.0
5,Colby ParkinsonC. Parkinson,DEN@SEA,10.0,0,0,0,0,0,0,0,...,0,2,43,1,0,0,0,SEA,2022.0,1.0
6,Zach ErtzZ. Ertz,KC@ARI,9.0,0,0,0,0,0,0,0,...,0,2,14,1,1,0,0,ARI,2022.0,1.0
7,Tyler ConklinT. Conklin,BAL@NYJ,7.0,0,0,0,0,0,0,0,...,0,4,14,1,0,0,0,NYJ,2022.0,1.0
8,Pat FreiermuthP. Freiermuth,PIT@CIN,7.0,0,0,0,0,0,0,0,...,0,5,75,0,0,0,0,PIT,2022.0,1.0
9,Darren WallerD. Waller,LV@LAC,7.0,0,0,0,0,0,0,0,...,0,4,79,0,0,0,0,LV,2022.0,1.0


In [40]:
# Clean TEs
df4 = df4.rename(columns={'Unnamed: 0_level_0_Player':'player', 'Unnamed: 1_level_0_Game':'opponent',
       'Passing_Att':'passA', 'Passing_Cmp':"passC", 'Passing_Yds':"passY",
       'Passing_TD':'passT', 'Passing_Int':'passI', 'Passing_2Pt':'pass2', 'Rushing_Att':'rushA',
       'Rushing_Yds':'rushY', 'Rushing_TD':'rushT', 'Rushing_2Pt':'rush2', 'Receiving_Rec':'recC',
       'Receiving_Yds':'recY', 'Receiving_TD':'recT', 'Receiving_2Pt':'rec2', 'Fumbles_FL':'fum', 'Fumbles_TD':'fumT'
       })
df4 = df4[['player', 'season', 'week', 'team', 'opponent', 'passA', 'passC', 'passY', 'passT', 'passI', 'pass2',
       'rushA', 'rushY', 'rushT', 'rush2', 'recC', 'recY', 'recT', 'rec2',
       'fum']]
df4[['tm1', 'tm2']] = df4['opponent'].str.split('@', expand=True)

def teamPicker(row):
    team = row['team']
    if row['tm1']==team:
        opp = row['tm2']
    else:
        opp = row['tm1']
    return opp
df4['opponent'] = df4.apply(teamPicker, axis=1)

df4 = df4.drop(columns=['tm1', 'tm2'])

df4[['player', 'todrop']] = df4['player'].str.split('.\xa0', expand=True)
df4 = df4.drop(columns=['todrop'])

def dropLast(name):
    name = name[:-1]
    return name
df4['player'] = df4['player'].apply(dropLast)

df4['pos'] = 'TE'

df4['posRank'] = 'NA'
for i in range(len(df4)):
    team = df4.iloc[i]['team']
    season = df4.iloc[i]['season']
    week = df4.iloc[i]['week']
    player = df4.iloc[i]['player']
    slicedf = df4.loc[(df4['team']==team) & (df4['season']==season) & (df4['week']==week)]
    slicedf = slicedf.sort_values(by=['recC', 'rushA'], ascending=False, ignore_index=True)
    posRank = slicedf.index[slicedf['player'] == player].to_list()[0]
    posRank += 1
    posRank = str(posRank)
    df4.at[i, 'posRank'] = posRank
df4['posRank'] = df4['pos'] + df4['posRank']
df4

,player,season,week,team,opponent,passA,passC,passY,passT,passI,...,rushY,rushT,rush2,recC,recY,recT,rec2,fum,pos,posRank
0,Travis Kelce,2022.0,1.0,KC,ARI,0,0,0,0,0,...,0,0,0,8,121,1,0,0,TE,TE1
1,O.J. Howard,2022.0,1.0,HOU,IND,0,0,0,0,0,...,0,0,0,2,38,2,0,0,TE,TE1
2,Taysom Hill,2022.0,1.0,NO,ATL,0,0,0,0,0,...,81,1,0,1,2,0,0,0,TE,TE2
3,Gerald Everett,2022.0,1.0,LAC,LV,0,0,0,0,0,...,0,0,0,3,54,1,0,0,TE,TE1
4,Will Dissly,2022.0,1.0,SEA,DEN,0,0,0,0,0,...,0,0,0,3,43,1,0,0,TE,TE1
5,Colby Parkinson,2022.0,1.0,SEA,DEN,0,0,0,0,0,...,0,0,0,2,43,1,0,0,TE,TE3
6,Zach Ertz,2022.0,1.0,ARI,KC,0,0,0,0,0,...,0,0,0,2,14,1,1,0,TE,TE1
7,Tyler Conklin,2022.0,1.0,NYJ,BAL,0,0,0,0,0,...,0,0,0,4,14,1,0,0,TE,TE1
8,Pat Freiermuth,2022.0,1.0,PIT,CIN,0,0,0,0,0,...,0,0,0,5,75,0,0,0,TE,TE1
9,Darren Waller,2022.0,1.0,LV,LAC,0,0,0,0,0,...,0,0,0,4,79,0,0,0,TE,TE1


In [12]:
# Scrape PKs
df=pd.DataFrame(columns=['Player', 'Game', 'Pts*', 'XPA', 'XPM', 'FGA', 'FGM', '50+'])
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

for j in range(1, 2):

    season = 2022
    week = j
    
    # Scrape web for stats
    url = f"https://www.footballdb.com/fantasy-football/index.html?pos=K&yr={season}&wk={week}&key=48ca46aa7d721af4d58dccc0c249a1c4"

    PATH = "/Applications/chromedriver"
    driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
    wait = WebDriverWait(driver, 20)
    driver.get(url)

    wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='leftcol']/div[3]/table")))
    driver.execute_script("window.stop();")

    result = pd.read_html(driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table").get_attribute("outerHTML"))
    regex_result = driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table/tbody").get_attribute("outerHTML")

    # find player's team based on which team is bolded
    regex_teams = re.findall("<b>(.*?)</b>", regex_result)

    # flatten multiindex
    result = result[0]
    #result.columns = result.columns.get_level_values(0) + '_' +  result.columns.get_level_values(1)

    # Set values for new columns
    result['team'] = regex_teams
    result['season'] = season
    result['week'] = week

    # concatenate to master df
    df = pd.concat([df, result], axis=0, ignore_index=True)

df5 = df

,Player,Game,Pts*,XPA,XPM,FGA,FGM,50+,team,season,week
0,Younghoe KooY. Koo,NO@ATL,18.0,2,2,5,4,2,ATL,2022.0,1.0
1,Cade YorkC. York,CLE@CAR,16.0,2,2,4,4,1,CLE,2022.0,1.0
2,Chris BoswellC. Boswell,PIT@CIN,13.0,2,2,4,3,1,PIT,2022.0,1.0
3,Greg JosephG. Joseph,GB@MIN,13.0,2,2,3,3,1,MIN,2022.0,1.0
4,Ryan SuccopR. Succop,TB@DAL,13.0,1,1,5,4,0,TB,2022.0,1.0
5,Brandon McManusB. McManus,DEN@SEA,10.0,1,1,4,3,0,DEN,2022.0,1.0
6,Riley PattersonR. Patterson,JAX@WAS,10.0,1,1,4,3,0,JAX,2022.0,1.0
7,Harrison ButkerH. Butker,KC@ARI,9.0,4,4,1,1,1,KC,2022.0,1.0
8,Daniel CarlsonD. Carlson,LV@LAC,9.0,1,1,2,2,1,LV,2022.0,1.0
9,Wil LutzW. Lutz,NO@ATL,9.0,1,1,3,2,1,NO,2022.0,1.0


In [20]:
# Clean PKs
df5 = df5.rename(columns={'Player':'player', 'Game':'opponent',
       '50+':'FG50'
       })
df5 = df5[['player', 'season', 'week', 'team', 'opponent', 'XPA', 'XPM', 'FGA', 'FGM', 'FG50'
       ]]

df5[['tm1', 'tm2']] = df5['opponent'].str.split('@', expand=True)

def teamPicker(row):
    team = row['team']
    if row['tm1']==team:
        opp = row['tm2']
    else:
        opp = row['tm1']
    return opp
df5['opponent'] = df5.apply(teamPicker, axis=1)

df5 = df5.drop(columns=['tm1', 'tm2'])

df5[['player', 'todrop']] = df5['player'].str.split('.\xa0', expand=True)
df5 = df5.drop(columns=['todrop'])

def dropLast(name):
    name = name[:-1]
    return name
df5['player'] = df5['player'].apply(dropLast)

df5['pos'] = 'PK'

df5['posRank'] = 'NA'
for i in range(len(df5)):
    team = df5.iloc[i]['team']
    season = df5.iloc[i]['season']
    week = df5.iloc[i]['week']
    player = df5.iloc[i]['player']
    slicedf = df5.loc[(df5['team']==team) & (df5['season']==season) & (df5['week']==week)]
    slicedf = slicedf.sort_values(by=['FGA', 'XPA'], ascending=False, ignore_index=True)
    posRank = slicedf.index[slicedf['player'] == player].to_list()[0]
    posRank += 1
    posRank = str(posRank)
    df5.at[i, 'posRank'] = posRank
df5['posRank'] = df5['pos'] + df5['posRank']

In [14]:
# Scrape Defs
df=pd.DataFrame(columns=['Team', 'Game', 'Pts*', 'Sack', 'Int', 'Saf', 'FR', 'Blk', 'TD', 'PA',
       'PassYds', 'RushYds', 'TotYds'])
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

for j in range(1, 2):

    season = 2022
    week = j
    
    # Scrape web for stats
    url = f"https://www.footballdb.com/fantasy-football/index.html?pos=DST&yr={season}&wk={week}&key=48ca46aa7d721af4d58dccc0c249a1c4"

    PATH = "/Applications/chromedriver"
    driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
    wait = WebDriverWait(driver, 20)
    driver.get(url)

    wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='leftcol']/div[3]/table")))
    driver.execute_script("window.stop();")

    result = pd.read_html(driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table").get_attribute("outerHTML"))
    regex_result = driver.find_element(By.XPATH, value="//div[@id='leftcol']/div[3]/table/tbody").get_attribute("outerHTML")

    # find player's team based on which team is bolded
    regex_teams = re.findall("<b>(.*?)</b>", regex_result)

    # flatten multiindex
    result = result[0]
    #result.columns = result.columns.get_level_values(0) + '_' +  result.columns.get_level_values(1)

    # Set values for new columns
    result['team'] = regex_teams
    result['season'] = season
    result['week'] = week

    # concatenate to master df
    df = pd.concat([df, result], axis=0, ignore_index=True)

df6 = df



,Team,Game,Pts*,Sack,Int,Saf,FR,Blk,TD,PA,PassYds,RushYds,TotYds,team,season,week
0,Pittsburgh SteelersPIT,PIT@CIN,29.0,7.0,4,0,1,1,1,20,299,133,432,PIT,2022.0,1.0
1,Miami DolphinsMIA,NE@MIA,21.0,2.0,1,0,2,0,1,7,193,78,271,MIA,2022.0,1.0
2,Buffalo BillsBUF,BUF@LA,20.0,7.0,3,0,0,0,0,10,191,52,243,BUF,2022.0,1.0
3,Los Angeles ChargersLAC,LV@LAC,16.0,6.0,3,0,0,0,0,19,256,64,320,LAC,2022.0,1.0
4,Tampa Bay BuccaneersTB,TB@DAL,16.0,4.0,1,0,0,0,0,3,173,71,244,TB,2022.0,1.0
5,Minnesota VikingsMIN,GB@MIN,15.0,4.0,1,0,1,0,0,7,227,111,338,MIN,2022.0,1.0
6,Baltimore RavensBAL,BAL@NYJ,14.0,3.0,1,0,1,0,0,9,295,83,378,BAL,2022.0,1.0
7,Chicago BearsCHI,SF@CHI,13.0,2.0,1,0,1,0,0,10,155,176,331,CHI,2022.0,1.0
8,Houston TexansHOU,IND@HOU,10.0,2.0,1,0,1,0,0,20,340,177,517,HOU,2022.0,1.0
9,Tennessee TitansTEN,NYG@TEN,10.0,5.0,1,0,1,0,0,21,156,238,394,TEN,2022.0,1.0


In [22]:
# Clean Defs
df6 = df6.rename(columns={'Team':'player', 'Game':'opponent', 'Sack':'defSack', 'Int':'defI', 'Saf':'defSaf', 'FR':'defFum', 
        'Blk':'defBlk','TD':'defT', 'PA':'defPtsAgainst', 'PassYds':'defPassYAgainst', 'RushYds':'defRushYAgainst', 'TotYds':'defYdsAgainst'
       })
df6 = df6[['player', 'season', 'week', 'team', 'opponent', 'defSack', 'defI', 'defSaf', 'defFum', 'defBlk',
       'defT', 'defPtsAgainst', 'defPassYAgainst', 'defRushYAgainst',
       'defYdsAgainst'
       ]]

df6[['tm1', 'tm2']] = df6['opponent'].str.split('@', expand=True)

def teamPicker(row):
    team = row['team']
    if row['tm1']==team:
        opp = row['tm2']
    else:
        opp = row['tm1']
    return opp
df6['opponent'] = df6.apply(teamPicker, axis=1)

df6 = df6.drop(columns=['tm1', 'tm2'])

df6['pos'] = 'DF'

df6['posRank'] = 'DF1'
df6

,player,season,week,team,opponent,defSack,defI,defSaf,defFum,defBlk,defT,defPtsAgainst,defPassYAgainst,defRushYAgainst,defYdsAgainst,pos,posRank
0,Pittsburgh SteelersPIT,2022.0,1.0,PIT,CIN,7.0,4,0,1,1,1,20,299,133,432,DF,DF1
1,Miami DolphinsMIA,2022.0,1.0,MIA,NE,2.0,1,0,2,0,1,7,193,78,271,DF,DF1
2,Buffalo BillsBUF,2022.0,1.0,BUF,LA,7.0,3,0,0,0,0,10,191,52,243,DF,DF1
3,Los Angeles ChargersLAC,2022.0,1.0,LAC,LV,6.0,3,0,0,0,0,19,256,64,320,DF,DF1
4,Tampa Bay BuccaneersTB,2022.0,1.0,TB,DAL,4.0,1,0,0,0,0,3,173,71,244,DF,DF1
5,Minnesota VikingsMIN,2022.0,1.0,MIN,GB,4.0,1,0,1,0,0,7,227,111,338,DF,DF1
6,Baltimore RavensBAL,2022.0,1.0,BAL,NYJ,3.0,1,0,1,0,0,9,295,83,378,DF,DF1
7,Chicago BearsCHI,2022.0,1.0,CHI,SF,2.0,1,0,1,0,0,10,155,176,331,DF,DF1
8,Houston TexansHOU,2022.0,1.0,HOU,IND,2.0,1,0,1,0,0,20,340,177,517,DF,DF1
9,Tennessee TitansTEN,2022.0,1.0,TEN,NYG,5.0,1,0,1,0,0,21,156,238,394,DF,DF1


In [72]:
offense = pd.concat([df1, df2, df3, df4], axis=0, ignore_index=True)
complete = pd.concat([offense, df5, df6], axis=0, ignore_index=True)
complete = complete[['season', 'week', 'team', 'player', 
       'pos', 'posRank', 'opponent', 
       'passA', 'passC','passY', 'passT', 'passI', 'pass2', 
       'rushA', 'rushY', 'rushT', 'rush2',
       'recC', 'recY', 'recT', 'rec2', 'fum', 
       'XPA', 'XPM', 'FGA', 'FGM', 'FG50', 
       'defSack', 'defI', 'defSaf', 'defFum', 'defBlk', 'defT', 'defPtsAgainst', 'defPassYAgainst', 'defRushYAgainst', 'defYdsAgainst']]
complete


,season,week,team,player,pos,posRank,opponent,passA,passC,passY,...,defSack,defI,defSaf,defFum,defBlk,defT,defPtsAgainst,defPassYAgainst,defRushYAgainst,defYdsAgainst
0,2022.0,1.0,KC,Patrick Mahomes,QB,QB1,ARI,39,30,360,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022.0,1.0,BUF,Josh Allen,QB,QB1,LA,31,26,297,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022.0,1.0,WAS,Carson Wentz,QB,QB1,JAX,41,27,313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022.0,1.0,LAC,Justin Herbert,QB,QB1,LV,34,26,279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022.0,1.0,BAL,Lamar Jackson,QB,QB1,NYJ,30,17,213,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,2022.0,1.0,CIN,Cincinnati BengalsCIN,DF,DF1,PIT,NaN,NaN,NaN,...,1.0,0,0,0,0,0,23,192,75,267
350,2022.0,1.0,ARI,Arizona CardinalsARI,DF,DF1,KC,NaN,NaN,NaN,...,0.0,0,0,1,0,0,44,360,128,488
351,2022.0,1.0,CAR,Carolina PanthersCAR,DF,DF1,CLE,NaN,NaN,NaN,...,1.0,0,0,0,0,0,26,138,217,355
352,2022.0,1.0,DET,Detroit LionsDET,DF,DF1,PHI,NaN,NaN,NaN,...,1.0,0,0,0,0,0,38,239,216,455


In [73]:
df = complete.copy()
# Clean defense names in historical data
# Create function to check if last three characters are uppercase. Strip only the uppercase
def cleanDefNames(text):
    if text[-3:].isupper():
        text = text[:-3]
    else:
        text = text[:-2]
    return(text)
# Apply the function to the defenses
df.loc[df.pos=='DF', 'player'] = df.loc[df.pos=='DF', 'player'].apply(cleanDefNames)

# Rename past teams to their most recent name to account for name changes or city moves
df.team.replace(to_replace=['STL', 'LA'], value='LAR', inplace=True)
df.player.replace(to_replace=['St. Louis Rams'], value='Los Angeles Rams', inplace=True)
df.team.replace(to_replace=['OAK', 'LV'], value='LVR', inplace=True)
df.player.replace(to_replace=['Oakland Raiders'], value='Las Vegas Raiders', inplace=True)
df.team.replace(to_replace=['NE'], value='NEP', inplace=True)
df.team.replace(to_replace=['SD'], value='LAC', inplace=True)
df.team.replace(to_replace=['TB'], value='TBB', inplace=True)
df.team.replace(to_replace=['NO'], value='NOS', inplace=True)
df.team.replace(to_replace=['GB'], value='GBP', inplace=True)
df.team.replace(to_replace=['KC'], value='KCC', inplace=True)
df.team.replace(to_replace=['SF'], value='SFO', inplace=True)
df.team.replace(to_replace=['TB'], value='TBB', inplace=True)

df.opponent.replace(to_replace=['STL', 'LA'], value='LAR', inplace=True)
df.opponent.replace(to_replace=['OAK', 'LV'], value='LVR', inplace=True)
df.opponent.replace(to_replace=['NE'], value='NEP', inplace=True)
df.opponent.replace(to_replace=['SD'], value='LAC', inplace=True)
df.opponent.replace(to_replace=['TB'], value='TBB', inplace=True)
df.opponent.replace(to_replace=['NO'], value='NOS', inplace=True)
df.opponent.replace(to_replace=['GB'], value='GBP', inplace=True)
df.opponent.replace(to_replace=['KC'], value='KCC', inplace=True)
df.opponent.replace(to_replace=['SF'], value='SFO', inplace=True)
df.opponent.replace(to_replace=['TB'], value='TBB', inplace=True)

In [74]:
# Set NaN values to zero
df.fillna(0, inplace=True)

df

,season,week,team,player,pos,posRank,opponent,passA,passC,passY,...,defSack,defI,defSaf,defFum,defBlk,defT,defPtsAgainst,defPassYAgainst,defRushYAgainst,defYdsAgainst
0,2022.0,1.0,KCC,Patrick Mahomes,QB,QB1,ARI,39,30,360,...,0.0,0,0,0,0,0,0,0,0,0
1,2022.0,1.0,BUF,Josh Allen,QB,QB1,LAR,31,26,297,...,0.0,0,0,0,0,0,0,0,0,0
2,2022.0,1.0,WAS,Carson Wentz,QB,QB1,JAX,41,27,313,...,0.0,0,0,0,0,0,0,0,0,0
3,2022.0,1.0,LAC,Justin Herbert,QB,QB1,LVR,34,26,279,...,0.0,0,0,0,0,0,0,0,0,0
4,2022.0,1.0,BAL,Lamar Jackson,QB,QB1,NYJ,30,17,213,...,0.0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,2022.0,1.0,CIN,Cincinnati Bengals,DF,DF1,PIT,0,0,0,...,1.0,0,0,0,0,0,23,192,75,267
350,2022.0,1.0,ARI,Arizona Cardinals,DF,DF1,KCC,0,0,0,...,0.0,0,0,1,0,0,44,360,128,488
351,2022.0,1.0,CAR,Carolina Panthers,DF,DF1,CLE,0,0,0,...,1.0,0,0,0,0,0,26,138,217,355
352,2022.0,1.0,DET,Detroit Lions,DF,DF1,PHI,0,0,0,...,1.0,0,0,0,0,0,38,239,216,455


In [75]:
df.columns

Index(['season', 'week', 'team', 'player', 'pos', 'posRank', 'opponent',
       'passA', 'passC', 'passY', 'passT', 'passI', 'pass2', 'rushA', 'rushY',
       'rushT', 'rush2', 'recC', 'recY', 'recT', 'rec2', 'fum', 'XPA', 'XPM',
       'FGA', 'FGM', 'FG50', 'defSack', 'defI', 'defSaf', 'defFum', 'defBlk',
       'defT', 'defPtsAgainst', 'defPassYAgainst', 'defRushYAgainst',
       'defYdsAgainst'],
      dtype='object')

In [76]:
# Rename all columns in df
colList = [(x + "_curr") for x in list(df.columns)]
df.columns = colList
df = df.rename(columns={'season_curr':'season',
 'week_curr':'week',
 'team_curr':'team',
 'player_curr':'player',
 'pos_curr':'pos',
 'posRank_curr':'posRank',
 'opponent_curr':'opponent'
})

In [77]:
#Analyze
analyzed = df.groupby(['player', 'pos']).mean()
analyzed['gamesPlayed'] = df.groupby(['player', 'pos'])['week'].count()
analyzed.drop(columns='week', inplace=True)
gamesPlayedCol = analyzed.pop('gamesPlayed')
analyzed.insert(0, 'gamesPlayed', gamesPlayedCol)
analyzed

,,gamesPlayed,season,passA_curr,passC_curr,passY_curr,passT_curr,passI_curr,pass2_curr,rushA_curr,rushY_curr,...,defSack_curr,defI_curr,defSaf_curr,defFum_curr,defBlk_curr,defT_curr,defPtsAgainst_curr,defPassYAgainst_curr,defRushYAgainst_curr,defYdsAgainst_curr
player,pos,,,,,,,,,,,,,,,,,,,,,
A.J. Brown,WR,1,2022.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A.J. Green,WR,1,2022.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AJ Dillon,RB,1,2022.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,45.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aaron Jones,RB,1,2022.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,49.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aaron Rodgers,QB,1,2022.0,34.0,22.0,195.0,0.0,1.0,0.0,1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zach Ertz,TE,1,2022.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zach Gentry,TE,1,2022.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zach Pascal,WR,1,2022.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
